In [23]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
import os
import keras
import math
import numpy as np

best_weight_path = "../model/best.h5"
best_acc = 0

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
#top_model_weights_path = '../model/bottleneck_fc_model.h5'
top_model_weights_path = '../model/bottleneck_fc_model_inceptionv3_500_val_acc_4791.h5'
#top_model_weights_path = '../model/fc_model.h5'
img_width, img_height = 500, 500

#full
train_data_dir = '../data/Pandora18K_train_val_test_split/train'
validation_data_dir = '../data/Pandora18K_train_val_test_split/val'
nb_train_samples = 14313
nb_validation_samples = 1772
total_num_classes = 18

#small
#train_data_dir = '../data/Pandora18K_small_train_val_test_split/train'
#validation_data_dir = '../data/Pandora18K_small_train_val_test_split/val'
#test_data_dir = '../data/Pandora18K_small_train_val_test_split/test'
#nb_train_samples = 1462
#nb_validation_samples = 167
#total_num_classes = 18

#verysmall
#train_data_dir = '../data/verysmall/train'
#validation_data_dir = '../data/verysmall/val'
#nb_train_samples = 144 #157
#nb_validation_samples = 16 #18
#total_num_classes = 2

epochs = 1
batch_size = 16

print("best_acc = " + str(best_acc))

best_acc = 0


In [24]:

# build the VGG16 network
base_model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3))
#base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
"""
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
#top_model.add(Dropout(0.5))
top_model.add(Dense(total_num_classes, activation='sigmoid'))
"""

"""
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dropout(0.5))
top_model.add(Dense(1024, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(total_num_classes, activation='softmax'))
"""
top_model = Sequential()
#top_model.add(Flatten(input_shape=train_data.shape[1:]))
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dropout(0.6))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(.6))
top_model.add(Dense(18, activation='softmax'))


# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
# model.add(top_model)

model = Model(input= base_model.input, output= top_model(base_model.output))



model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
              metrics=['accuracy'])




# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
valid_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen  = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_gen = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')




_, sanity_acc = model.evaluate_generator(generator = validation_generator, steps = nb_validation_samples // batch_size)
print("sanity_acc = " + str(sanity_acc))


"""
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')
"""

Model loaded.
Found 14313 images belonging to 18 classes.
Found 1772 images belonging to 18 classes.
Found 171 images belonging to 18 classes.
sanity_acc = 0.515909090909


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:41: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`


"\n# prepare data augmentation configuration\ntrain_datagen = ImageDataGenerator(\n    rescale=1. / 255,\n    shear_range=0.2,\n    zoom_range=0.2,\n    horizontal_flip=True)\n\ntest_datagen = ImageDataGenerator(rescale=1. / 255)\n\ntrain_generator = train_datagen.flow_from_directory(\n    train_data_dir,\n    target_size=(img_height, img_width),\n    batch_size=batch_size,\n    class_mode='categorical')\n\nvalidation_generator = test_datagen.flow_from_directory(\n    validation_data_dir,\n    target_size=(img_height, img_width),\n    batch_size=batch_size,\n    class_mode='categorical')\n"

In [12]:
"""
class AccuracyHistory(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        
        global best_acc
        current_acc = logs.get('val_acc')
        
        if current_acc > best_acc:
            print("\n\na better current_acc = " + str(current_acc) + "\n\n")
            best_acc = current_acc
            model.save_weights(best_weight_path,overwrite=True)
"""

def run_with_parameters(frozen_layers = 25, lr = 1e-4, momentum = 0.9):
    global best_acc
    global best_weight_path
    print("best val acc so far = " + str(best_acc))
    
    # set the first 25 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
    for layer in model.layers:
        layer.trainable = True;
    
    for layer in model.layers[:frozen_layers]:
        layer.trainable = False

    # compile the model with a SGD/momentum optimizer
    # and a very slow learning rate.
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=lr, momentum=momentum),
                  metrics=['accuracy'])
    
    pretrained_model = "temp_name"
    weight_dir = "../model/%s"%pretrained_model
    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)
    param_names = "frozen_layers_" + str(frozen_layers) + "_lr_" + str(lr) + "_momentum_" + str(momentum)
    weight_path = os.path.join(weight_dir, param_names + "_best_weights%s.h5"%pretrained_model)
    checkpointer = keras.callbacks.ModelCheckpoint(filepath=weight_path, verbose=1,monitor='val_acc', save_best_only=True, save_weights_only=True)

    model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks=[checkpointer])
    
    #model.load_weights(weight_path)
    _, sanity_acc = model.evaluate_generator(generator = validation_generator, steps = nb_validation_samples // batch_size)
    
    print("sanity_acc = "+ str(sanity_acc))
    
    if sanity_acc > best_acc:
        print("updating best_acc by loading weights from " + weight_path)
        model.load_weights(weight_path)
        model.save_weights(best_weight_path, overwrite = True)
        best_acc = sanity_acc
    
    print("best val acc so far = " + str(best_acc))

In [13]:

frozen_layers_arr = [25]
lr_arr = [1e-4, 1e-5, 1e-2, 1e-6]
momentum_arr = [0.9]
shear_range_arr = [0.2]
zoom_range_arr = [0.2]
horizontal_flip_arr = [True]

#0:lr,1:dense middle,2:dropout1, 3:dropout2
#params=(10**np.random.uniform(low=-5, high=-1),np.random.randint(low = 50, high=800),np.random.uniform(low=0.3,high=0.8), np.random.uniform(low=0.3,high=0.8))

for i in range(5):
    frozen_layers_val = 25#np.random.randint(low = 25, high=25)
    lr_val = 10**np.random.uniform(low=-5, high=-1)
    print("\n")
    print("lr_val = " + str(lr_val))

    run_with_parameters(frozen_layers = frozen_layers_val, lr = lr_val)
            

print("\n\n\nbest val acc = " + str(best_acc))
    



lr_val = 0.000207014464829
best val acc so far = 0
Epoch 1/1
91/91 [==============================] - 16s - loss: 2.8043 - acc: 0.1676 - val_loss: 2.6641 - val_acc: 0.1875
sanity_acc = 0.192052980725
updating best_acc by loading weights from ../model/temp_name/frozen_layers_25_lr_0.000207014464829_momentum_0.9_best_weightstemp_name.h5
best val acc so far = 0.192052980725


lr_val = 0.0362757856317
best val acc so far = 0.192052980725
Epoch 1/1
91/91 [==============================] - 16s - loss: 2.7772 - acc: 0.1653 - val_loss: 2.6919 - val_acc: 0.1921
sanity_acc = 0.21854304695
updating best_acc by loading weights from ../model/temp_name/frozen_layers_25_lr_0.0362757856317_momentum_0.9_best_weightstemp_name.h5
best val acc so far = 0.21854304695


lr_val = 0.000679879756745
best val acc so far = 0.21854304695
Epoch 1/1
91/91 [==============================] - 16s - loss: 2.8052 - acc: 0.1662 - val_loss: 2.6245 - val_acc: 0.1987
sanity_acc = 0.185430463576
best val acc so far = 0.218

In [14]:
#sanity check that the saved best_weight_path contains weights that gets you an accuracy of best_acc
#sanity_val_datagen = ImageDataGenerator(rescale=1. / 255)
#sanity_val_generator = sanity_val_datagen.flow_from_directory(
#    validation_data_dir,
#    target_size=(img_width, img_height),
#    batch_size=batch_size,
#    class_mode='categorical')

model.load_weights(best_weight_path)

sanity_loss, sanity_acc = model.evaluate_generator(generator = validation_generator, steps = (nb_validation_samples // batch_size)+1)
print("best val loss check = " + str(sanity_loss))
print("best val acc check = " + str(sanity_acc))

best val loss check = 2.66694613131
best val acc check = 0.179640718563
